In [21]:
import sys

origin = 'data/Aldhani/eoagritwin/'
sys.path.append('/home/potzschf/repos/')

from helperToolz.polygons_to_labels import *
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import * 

gtiff_driver = gdal.GetDriverByName('GTiff')

In [23]:
# get the Brandenburg 2019 comps for test run (take away the IACS data)
outFolder = path_safe('/data/Aldhani/eoagritwin/fields/output/trash/for_Shreya/')
force_tifs = getFilelist('/data/Aldhani/eoagritwin/force/output/BRB/2019/X0067_Y0041/FBM/', '.tif')

ds= gdal.Open(force_tifs[0])
XX = ds.RasterXSize
YY = ds.RasterYSize
gt = ds.GetGeoTransform()
prj = ds.GetProjection()

In [ ]:
for month in range(3,9,1):
    monthly = [tif for tif in force_tifs if f'-{month:02d}.tif' in tif]
    MM = (INT_TO_MONTH[f'{month:02d}'])
    # export labels as tif chips
    out_ds = gtiff_driver.Create((f'{outFolder}{MM}.tif'), XX, YY, 4, gdal.GDT_Float32)

    out_ds.SetGeoTransform(gt)
    out_ds.SetProjection(prj)

    

    for band, color in enumerate['BLU', 'GRN', 'RED', 'BNR']:
        for file in monthly:
            if color == file.split('_SEN2H_')[-1].split('_FBM')[0]:
                dum_ds = gdal.Open(file)
                arr = dum_ds.GetRasterBand(1).ReadAsArray()
                out_ds.GetRasterBand(band + 1).WriteArray(arr)
                out_ds.GetRasterBand(band + 1).SetDescription(color)

    del out_ds

In [29]:
prefix = '/data/Aldhani/eoagritwin/'
state = 'SL'
year = 2021

f"{prefix}/fields/Auxiliary/vrt/{state}/{year}/{dirfinder(f'{prefix}/fields/Auxiliary/vrt/{state}/{year}/')[0]}"

'/data/Aldhani/eoagritwin//fields/Auxiliary/vrt/SL/2021/Force_X_from_53_to_55_Y_from_54_to_56'